In [422]:
import json
import numpy as np
import plotly.express as px
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, ConstantKernel
import scipy
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
def optimizer(obj_func, initial_theta, bounds):
    opt_res = scipy.optimize.minimize(obj_func, initial_theta, method="L-BFGS-B", jac=True,bounds=bounds,options={"maxiter":10000})
    theta_opt, func_min = opt_res.x, opt_res.fun
    return theta_opt, func_min

In [440]:
from scipy.stats import sobol_indices, uniform
rng = np.random.default_rng()

# sobol indices tell you how much of the variance in output is from variance in inputs
def get_sobol_indices(inputs, input_names, gp):
    def sampling(X):
        n = np.shape(X)[1]
        d = np.array([gp.predict([X[:, i]])[0] for i in range(0, n)])
        return d

    dists = []
    n_inputs = np.shape(inputs)[1]
    for i in range(0, n_inputs):
        d = [k[i] for k in inputs]
        x = np.mean(d)
        sc = (np.max(d) - np.min(d)) / 2
        dists.append(uniform(loc=x, scale=sc))

    indices = sobol_indices(
        func=sampling,
        n=4096,
        dists=dists,
        random_state=rng,
    )

    return {
        input_names[i]: indices.first_order[i]
        for i in range(0, len(indices.first_order))
    }

In [469]:
# read in output data

costs_sb_hi =pd.read_csv("la100_data/CumulativeAnnualSystemCosts_SB100-High.csv").set_index("Year")
costs_sb_mod =pd.read_csv("la100_data/CumulativeAnnualSystemCosts(2021-2045)SB100-Moderate-Chart.csv").set_index("Year")
costs_bio_hi = pd.read_csv("la100_data/CumulativeAnnualSystemCosts_Early&NoBiofuels-High.csv").set_index("Year")
costs_bio_mod = pd.read_csv("la100_data/CumulativeAnnualSystemCosts(2021-2045)Early&NoBiofuels-Moderate-Chart.csv").set_index("Year")

jobs_sb_hi = pd.read_csv("la100_data/AnnualEmploymentbasedonInvestmentsbyJobCategoryandJobTypeSB100-High.csv").set_index("Year")
jobs_sb_mod = pd.read_csv("la100_data/AnnualEmploymentbasedonInvestmentsbyJobCategoryandJobTypeSB100-Moderate-Chart.csv").set_index("Year")
jobs_bio_hi = pd.read_csv("la100_data/AnnualEmploymentbasedonInvestmentsbyJobCategoryandJobTypeEarly&NoBiofuels-High.csv").set_index("Year")
jobs_bio_mod = pd.read_csv("la100_data/AnnualEmploymentbasedonInvestmentsbyJobCategoryandJobTypeEarly&NoBiofuels-Moderate-Chart.csv").set_index("Year")

emiss_sb_hi = pd.read_csv("la100_data/AnnualLifeCycleGHGEmissionsforthePowerandNon-PowerSectorsbySectorSB100-High.csv").set_index("Year")
emiss_sb_mod = pd.read_csv("la100_data/AnnualLifeCycleGHGEmissionsforthePowerandNon-PowerSectorsbySectorSB100-Moderate-Chart.csv").set_index("Year")
emiss_bio_hi = pd.read_csv("la100_data/AnnualLifeCycleGHGEmissionsforthePowerandNon-PowerSectorsbySectorEarly&NoBiofuels-High.csv").set_index("Year")
emiss_bio_mod = pd.read_csv("la100_data/AnnualLifeCycleGHGEmissionsforthePowerandNon-PowerSectorsbySectorEarly&NoBiofuels-Moderate-Chart.csv").set_index("Year")

12

In [472]:
# process output data to be nicely formatted

def proc_out(o):
    return [i for i in o.sum(axis=1).to_numpy()]

outputs_sb = {"emiss":proc_out(emiss_sb_hi)+proc_out(emiss_sb_mod),"jobs":proc_out(jobs_sb_hi)+proc_out(jobs_sb_mod),"costs":proc_out(costs_sb_hi)+proc_out(costs_sb_mod)}
outputs_bio = {"emiss":proc_out(emiss_bio_hi)+proc_out(emiss_bio_mod),"jobs":proc_out(jobs_bio_hi)+proc_out(jobs_bio_mod),"costs":proc_out(costs_bio_hi)+proc_out(costs_bio_mod)}

In [ ]:
# read in input data
load_data_hi =  pd.read_csv("la100_data/high_load_data.csv").set_index("Year")
load_data_mod =  pd.read_csv("la100_data/mod_load_data.csv").set_index("Year")

cap_sb_hi = pd.read_csv("la100_data/GenerationCapacityMixbyTechnologyTypeSB100-High.csv").set_index("Year")
cap_sb_mod = pd.read_csv("la100_data/GenerationCapacityMixbyTechnologyTypeSB100-Moderate-Chart.csv").set_index("Year")
cap_bio_hi = pd.read_csv("la100_data/GenerationCapacityMixbyTechnologyTypeEarly&NoBiofuels-High.csv").set_index("Year")
cap_bio_mod = pd.read_csv("la100_data/GenerationCapacityMixbyTechnologyTypeEarly&NoBiofuels-Moderate-Chart.csv").set_index("Year")

sroof_sb_hi = pd.read_csv("la100_data/CumulativeRooftopSolarAdoptionSB100-High.csv").set_index("Year")
sroof_sb_mod = pd.read_csv("la100_data/CumulativeRooftopSolarAdoptionSB100-Moderate-Chart.csv").set_index("Year")
sroof_bio_hi = pd.read_csv("la100_data/CumulativeRooftopSolarAdoptionEarly&NoBiofuels-High.csv").set_index("Year")
sroof_bio_mod = pd.read_csv("la100_data/CumulativeRooftopSolarAdoptionEarly&NoBiofuels-Moderate-Chart.csv").set_index("Year")

cstore_sb_hi = pd.read_csv("la100_data/CumulativeCustomerAdoptedStoragebySectorSB100-High-Chart.csv").set_index("Year")
cstore_sb_mod = pd.read_csv("la100_data/CumulativeCustomerAdoptedStoragebySectorSB100-Moderate-Chart.csv").set_index("Year")
cstore_bio_hi = pd.read_csv("la100_data/CumulativeCustomerAdoptedStoragebySectorEarly&NoBiofuels-High-Chart.csv").set_index("Year")
cstore_bio_mod = pd.read_csv("la100_data/CumulativeCustomerAdoptedStoragebySectorEarly&NoBiofuels-Moderate-Chart.csv").set_index("Year")

In [480]:
# define input matrix
def get_inputs(load,cap,roof,store):
    # load_total,cap_total
    return np.transpose(np.vstack((cap,load,roof,store)))
    
    
def proc_input(da_input1,da_input2,n):
    return [i for i in da_input1.sum(axis=1).to_numpy()][n:] +  [i for i in da_input2.sum(axis=1).to_numpy()][n:] 

In [503]:
# for input processing 
da_n_years = {"emiss":len(proc_out(emiss_sb_hi)),"jobs":len(proc_out(jobs_sb_hi)),"costs":len(proc_out(costs_sb_hi))}

In [521]:
# specify which output to build a gp for
oidx = "jobs"
input_names=["total capacity","total load","customer rooftop solar","customer storage"]# for sobol indices calc and plotting

In [522]:
# compute gp for the Bio scenario

output_bio = outputs_bio[oidx]
n_years = da_n_years[oidx]
inputs_bio = get_inputs(proc_input(load_data_hi,load_data_mod,6-n_years),proc_input(cap_bio_hi,cap_bio_mod,6-n_years),proc_input(sroof_bio_hi,sroof_bio_mod,6-n_years),proc_input(cstore_bio_hi,cstore_bio_mod,6-n_years))
kernel_bio = DotProduct(sigma_0=1.0,sigma_0_bounds=(1e-100, 100000.0)) + WhiteKernel(noise_level=1.0,noise_level_bounds=(1e-100, 100000.0))


gp_bio = GaussianProcessRegressor(
        kernel=kernel_bio,
        copy_X_train=False,
        normalize_y=True,
        n_restarts_optimizer=10,
        optimizer=optimizer
    ).fit(inputs_bio, output_bio)

print(gp_bio.score(inputs_bio,output_bio))
print(gp_bio.kernel_)
vals_bio,svals_bio = gp_bio.predict(inputs_bio[0:n_years],return_std=True)

sobol_bio = get_sobol_indices(inputs_bio,input_names,gp_bio)

0.8028893410462672
DotProduct(sigma_0=0.999) + WhiteKernel(noise_level=0.386)


In [523]:
# compute gp for the sb100 scenario
output_sb = outputs_sb[oidx]
n_years = da_n_years[oidx]
inputs_sb = get_inputs(proc_input(load_data_hi,load_data_mod,6-n_years),proc_input(cap_sb_hi,cap_sb_mod,6-n_years),proc_input(sroof_sb_hi,sroof_sb_mod,6-n_years),proc_input(cstore_sb_hi,cstore_sb_mod,6-n_years))
kernel_sb = DotProduct(sigma_0=1.0,sigma_0_bounds=(1e-100, 100000.0)) + WhiteKernel(noise_level=1.0,noise_level_bounds=(1e-100, 100000.0))


gp_sb = GaussianProcessRegressor(
        kernel=kernel_sb,
        copy_X_train=False,
        normalize_y=True,
        n_restarts_optimizer=10,
        optimizer=optimizer
    ).fit(inputs_sb, output_sb)

print(gp_sb.score(inputs_sb,output_sb))
print(gp_sb.kernel_)
vals_sb,svals_sb = gp_sb.predict(inputs_sb[0:n_years],return_std=True)

sobol_sb = get_sobol_indices(inputs_sb,input_names,gp_sb)

0.5583751401610859
DotProduct(sigma_0=0.00947) + WhiteKernel(noise_level=0.829)


In [525]:

# plot actual output and gp predictions

import plotly.graph_objects as go
x = list(load_data_hi.index[(6-n_years):].to_numpy())
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=vals_sb,error_y={"array":svals_sb},name="prediction sb"))
fig.add_trace(go.Scatter(x=x,y=output_sb,mode="markers",name="actual sb"))
fig.add_trace(go.Scatter(x=x, y=vals_bio,error_y={"array":svals_bio},name="prediction bio"))
fig.add_trace(go.Scatter(x=x,y=output_bio,mode="markers",name="actual bio"))
fig.update_layout(
    title=dict(text=oidx)
)

fig.show()

In [526]:

# plot sobol indices

fig = go.Figure()
xx = [i for i in range(0,len(sobol_bio))]
yy = [i for i in sobol_bio.values()]
fig.add_trace(go.Scatter(x=xx,y=yy,name="bio"))

xx = [i for i in range(0,len(sobol_sb))]
yy = [i for i in sobol_sb.values()]
fig.add_trace(go.Scatter(x=xx,y=yy,name="sb"))
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = xx,
        ticktext = [i for i in sobol_bio.keys()]
    ),
    yaxis = dict(range=(0,1),tickvals=np.linspace(0,1,11)),
    title=dict(text=oidx+" sobol indices")
)
fig.show()